In [7]:
!pip install pyDOE
!pip install cma

In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F3 '''
def F3(X):
    f = bn.F3()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 1.03 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 23.95 , C = 992.98 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_3_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_3_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [11]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [12]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(10,20):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-10,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 10
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=1034486, Wed Apr 15 14:03:50 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.934230180132861e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.793062302186601e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [21101.20578408] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [20066.68976723] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.888111308420566e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.4
   28    532 4.264398741490273e+03 1.1e+00 1.98e+00  2e+00  2e+00 0:03.4
   61   1159 5.613542615086291e+03 1.1e+00 1.74e+00  2e+00  2e+00 0:07.5
  100   1900 -9.470933962980926e+02 1.2e+00 1.40e+00  1e+00  1e+00 0:12.4
  150   2850 -1.143034676409065e+04 1.2e+00 9.82e-01  1e+00  1e+00 0:18.5
  200   3800 -2.088499754147784e+04 1.2e+00 5.76e-01  6e-01  6e-01 0:24.6
  266   5054 -2.652464018810822e+04 1.2e+00 3.03e-01  3e-01  3e-01 0:32.6
  300   5700 -2.743513985452258e+04 1.2e+00 2.13e-01  2e-01  2e-01 0:36.7
  381   7239 -2.828424190487165e+04 1.2e+00 9.25e-02  9e-02  9e-02 0:46.7
  400   7600 -2.834836148801477e+04 1.3e+00 7.56e-02  7e-02  8e-02 0:49.0
  498   9462 -2.850810644621891e+04 1.3e+00 2.79e-02  3e-02  3e-02 1:01.1
  500   9500 -2.850904015059242e+04 1.3e+00 2.74e-02  3e-02  3e-02 1:01.3
  600  11400 -2.852618357461812e+04 1.3e+00 1.04e-02  1e-02  1e-02 1:15.1
  700  13300 -2.852869322407063e+04 1.3e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [22322.25886811] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=22322.258868, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=22322.258868, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

termination on tolfun=1e-11 (Wed Apr 15 14:27:06 2020)
final/bestever f-value = 2.232226e+04 2.232226e+04
incumbent solution: [-3.66797892  2.38396844  1.93523925 -1.17954024  2.84159977  4.84165633
  0.92653047 -1.67758373 ...]
std deviations: [2.43649454 2.43627612 2.43633262 2.4363476  2.4363929  2.43615269
 2.43646416 2.4363316  ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=1040281, Wed Apr 15 14:27:06 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.739353997992122e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.485293244968766e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


...

   22    418 7.925933828967656e+03 1.0e+00 2.01e+00  2e+00  2e+00 0:03.6
   47    893 5.384409978570431e+03 1.1e+00 1.83e+00  2e+00  2e+00 0:07.7
   78   1482 -1.021725466410342e+03 1.1e+00 1.53e+00  2e+00  2e+00 0:12.7
  100   1900 -4.715173230705295e+03 1.2e+00 1.24e+00  1e+00  1e+00 0:16.3
  144   2736 -1.459753263967080e+04 1.2e+00 8.26e-01  8e-01  8e-01 0:23.4
  194   3686 -2.173453519507292e+04 1.2e+00 5.12e-01  5e-01  5e-01 0:31.5
  200   3800 -2.230222618123343e+04 1.2e+00 4.83e-01  5e-01  5e-01 0:32.5
  262   4978 -2.687051618680459e+04 1.2e+00 2.59e-01  3e-01  3e-01 0:42.5
  300   5700 -2.760773930445943e+04 1.2e+00 1.81e-01  2e-01  2e-01 0:48.6
  374   7106 -2.831554630252662e+04 1.2e+00 8.56e-02  8e-02  9e-02 1:00.7
  400   7600 -2.839422948200053e+04 1.2e+00 7.02e-02  7e-02  7e-02 1:05.0
  486   9234 -2.850715817486781e+04 1.2e+00 2.92e-02  3e-02  3e-02 1:19.0
  500   9500 -2.851436045365409e+04 1.2e+00 2.51e-02  2e-02  3e-02 1:21.3
  600  11400 -2.852676509020362e+04 1.2e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [22322.25886811] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=22322.258868, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=22322.258868, sigma=2.44e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

termination on tolfun=1e-11 (Wed Apr 15 20:14:10 2020)
final/bestever f-value = 2.232226e+04 2.232226e+04
incumbent solution: [-1.25197957 -4.94993332  1.74491608  4.90996113  3.56750237 -3.74900643
  3.65739347 -3.79035572 ...]
std deviations: [2.43549465 2.43521649 2.43562151 2.43534055 2.43558325 2.43537301
 2.43522154 2.43553598 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=1122431, Wed Apr 15 20:14:10 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.856485471573573e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.594690214870665e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [26448.96425978] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [21033.51771141] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.452802029760634e+04 1.0e+00 2.33e+00  2e+00  2e+00 0:00.3
   31    589 6.745211684443446e+03 1.1e+00 1.86e+00  2e+00  2e+00 0:03.4
   68   1292 5.770867433303298e+03 1.1e+00 1.69e+00  2e+00  2e+00 0:07.5
  100   1900 3.944971466250307e+03 1.2e+00 1.51e+00  1e+00  2e+00 0:11.1
  154   2926 1.760711203550054e+03 1.2e+00 1.20e+00  1e+00  1e+00 0:17.1
  200   3800 -1.419513816543345e+03 1.3e+00 9.72e-01  1e+00  1e+00 0:22.2
  272   5168 -3.551559518064336e+03 1.3e+00 6.95e-01  7e-01  7e-01 0:30.3
  300   5700 -4.803126167809012e+03 1.3e+00 5.88e-01  6e-01  6e-01 0:33.3
  390   7410 -7.173182201616472e+03 1.3e+00 3.48e-01  3e-01  4e-01 0:43.4
  400   7600 -7.270257610935263e+03 1.3e+00 3.31e-01  3e-01  3e-01 0:44.6
  500   9500 -8.448213577886832e+03 1.3e+00 2.04e-01  2e-01  2e-01 0:55.8
  600  11400 -8.923615869737921e+03 1.4e+00 1.32e-01  1e-01  1e-01 1:06.9
  700  13300 -9.230923542551078e+03 1.5e+00 8.74e-02  8e-02  1e-01 1:18.1
  800  15200 -9.331402650772347e+03 1.6e+00

In [14]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F3_X_Values_second.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F3(X_Values.iloc[i,:200])
    SVM_Fun [i] = F3(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F3(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([4292.73421439]), count=array([1])) 8.471032683537962e-05
SVM
ModeResult(mode=array([15471.72525769]), count=array([1])) 5095.7711160395265
ELN
ModeResult(mode=array([10076.55758687]), count=array([1])) 526.8567490360688
